# Retail Sales Analysis: Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Load the Retail dataset
Transactions_df = pd.read_csv('Online Retail - Raw Data.csv')
Transactions_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-10 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,09-12-11 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,09-12-11 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,09-12-11 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,09-12-11 12:50,4.15,12680.0,France


In [2]:
Transactions_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## **1. InvoiceNo**

In [3]:
# Missing values? 
Transactions_df[Transactions_df['InvoiceNo'].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [4]:
# Create a new column to mark cancelled transactions
Transactions_df['Cancelled'] = Transactions_df['InvoiceNo'].str.startswith('C')

# Convert InvoiceNo to numeric for non-cancelled rows
Transactions_df['InvoiceNo'] = Transactions_df['InvoiceNo'].str.extract('(\d+)').astype(int)

Transactions_df[(Transactions_df['Cancelled'] == True)]  

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled
141,536379,D,Discount,-1,01-12-10 9:41,27.50,14527.0,United Kingdom,True
154,536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,01-12-10 9:49,4.65,15311.0,United Kingdom,True
235,536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,01-12-10 10:24,1.65,17548.0,United Kingdom,True
236,536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,01-12-10 10:24,0.29,17548.0,United Kingdom,True
237,536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,01-12-10 10:24,0.29,17548.0,United Kingdom,True
...,...,...,...,...,...,...,...,...,...
540449,581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,09-12-11 9:57,0.83,14397.0,United Kingdom,True
541541,581499,M,Manual,-1,09-12-11 10:28,224.69,15498.0,United Kingdom,True
541715,581568,21258,VICTORIAN SEWING BOX LARGE,-5,09-12-11 11:57,10.95,15311.0,United Kingdom,True
541716,581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,09-12-11 11:58,1.25,17315.0,United Kingdom,True


## **2. StockCode**

In [5]:
# Missing values? 
Transactions_df[Transactions_df['StockCode'].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled


In [6]:
import re 

# Function to detect valid StockCode pattern (5 digits followed by 0 to 2 case insensitive letters)
valid_stockcode_pattern = r'^\d{5}[a-zA-Z]{0,2}$'
def is_valid_stockcode(stockcode):
    return bool(re.match(valid_stockcode_pattern, stockcode))

Transactions_df['ValidStockCode'] = Transactions_df['StockCode'].apply(is_valid_stockcode)

invalid_stockcode_rows = Transactions_df[Transactions_df['ValidStockCode'] == False]

print("invalid StockCode values:")
invalid_stockcode_rows[['Description', 'StockCode']].groupby(['Description', 'StockCode']).size().reset_index(name='Count')  

invalid StockCode values:


,Description,StockCode,Count
0,AMAZON FEE,AMAZONFEE,34
1,Adjust bad debt,B,3
2,BOXED GLASS ASHTRAY,DCGS0003,4
3,BOYS PARTY BAG,DCGSSBOY,11
4,Bank Charges,BANK CHARGES,37
5,CAMOUFLAGE DOG COLLAR,DCGS0070,1
6,CARRIAGE,C2,143
7,CRUK Commission,CRUK,16
8,DOTCOM POSTAGE,DOT,709
9,Discount,D,77


In [7]:
# Above shows the following values are valid too
specific_stockcodes = [
    "DCGSSBOY", "DCGS0070", "DCGSSGIRL", 
    "DCGS0004", "PADS", "DCGS0076"
]

# Update 'ValidStockCode' column 
Transactions_df['ValidStockCode'] = Transactions_df['ValidStockCode'] | \
                                     Transactions_df['StockCode'].isin(specific_stockcodes)

# Keep only rows with valid StockCodes
Transactions_df = Transactions_df[Transactions_df['ValidStockCode']].drop(columns=['ValidStockCode']).reset_index(drop=True)

## **3. Quantity**

In [8]:
# Missing values, or zeros?
Transactions_df[(Transactions_df['Quantity'].isnull()) 
        | (Transactions_df['Quantity'] == 0)] 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled


In [9]:
# Filter rows where Quantity is negative and Cancelled is False
negative_quantity_rows = Transactions_df[
    (Transactions_df['Quantity'] < 0) & 
    (~Transactions_df['Cancelled'])  # Not Cancelled
]

print(negative_quantity_rows)

print("Number of unique customers with negative quantities and not cancelled:", negative_quantity_rows['CustomerID'].nunique())
print("Number of unique UnitPrice with negative quantities and not cancelled:", negative_quantity_rows['UnitPrice'].nunique())

        InvoiceNo StockCode Description  Quantity     InvoiceDate  UnitPrice  \
2398       536589     21777         NaN       -10  01-12-10 16:50        0.0   
4338       536764    84952C         NaN       -38  02-12-10 14:42        0.0   
7164       536996     22712         NaN       -20  03-12-10 15:30        0.0   
7165       536997     22028         NaN       -20  03-12-10 15:30        0.0   
7166       536998     85067         NaN        -6  03-12-10 15:30        0.0   
...           ...       ...         ...       ...             ...        ...   
532388     581210     23395       check       -26  07-12-11 18:36        0.0   
532390     581212     22578        lost     -1050  07-12-11 18:38        0.0   
532391     581213     22576       check       -30  07-12-11 18:38        0.0   
533960     581226     23090     missing      -338   08-12-11 9:56        0.0   
535966     581422     23169     smashed      -235  08-12-11 15:24        0.0   

        CustomerID         Country  Can

In [10]:
# Above shows all the negative Quantities with (Cancel = False) have NaN CustomerID value and (UnitPrice = 0). Let's remove these rows. 
Transactions_df = Transactions_df[
    ~((Transactions_df['Quantity'] < 0) & (~Transactions_df['Cancelled']))
]

## **4. InvoiceDate**

In [11]:
# Convert to datetime
Transactions_df['InvoiceDate'] = pd.to_datetime(Transactions_df['InvoiceDate'], format='%d-%m-%y %H:%M', errors='coerce')

# Missing values or non-date, or zeros?
Transactions_df[Transactions_df['InvoiceDate'].isnull()] 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled


## **5. CustomerID**

In [12]:
# Missing values? 
print("It's okay to have missing values in CustomerID as long as InvoiceNo and StockCode are not missing.\n")
Transactions_df[Transactions_df['CustomerID'].isnull()]

It's okay to have missing values in CustomerID as long as InvoiceNo and StockCode are not missing.



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled
619,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom,False
1438,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom,False
1439,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom,False
1440,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom,False
1441,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom,False
...,...,...,...,...,...,...,...,...,...
538578,581498,85049e,SCANDINAVIAN REDS RIBBONS,4,2011-12-09 10:26:00,3.29,NaN,United Kingdom,False
538579,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom,False
538580,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom,False
538581,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom,False


In [13]:
# Replace missing CustomerIDs with "UNKNOWN" and remove floating point
Transactions_df['CustomerID'] = Transactions_df['CustomerID'].fillna('UNKNOWN').astype(str).str.replace(r'\.0$', '', regex=True)

## **6. UnitPrice**

In [14]:
# Missing values or negatives?
Transactions_df[(Transactions_df['UnitPrice'].isnull()) 
        | (Transactions_df['UnitPrice'] < 0)] 
        

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled


In [15]:
# Display all rows
pd.set_option('display.max_rows', None) 

In [16]:
# Identify rows where 'CustomerID' is not 'UNKNOWN' and 'UnitPrice' is zero (free items)
Transactions_df[(Transactions_df['CustomerID'] != 'UNKNOWN') & (Transactions_df['UnitPrice'] == 0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled
9269,537197,22841,ROUND CAKE TIN VINTAGE GREEN,1,2010-12-05 14:02:00,0.0,12647,Germany,False
33445,539263,22580,ADVENT CALENDAR GINGHAM SACK,4,2010-12-16 14:36:00,0.0,16560,United Kingdom,False
39917,539722,22423,REGENCY CAKESTAND 3 TIER,10,2010-12-21 13:45:00,0.0,14911,EIRE,False
46844,540372,22090,PAPER BUNTING RETROSPOT,24,2011-01-06 16:41:00,0.0,13081,United Kingdom,False
46846,540372,22553,PLASTERS IN TIN SKULLS,24,2011-01-06 16:41:00,0.0,13081,United Kingdom,False
56409,541109,22168,ORGANISER WOOD ANTIQUE WHITE,1,2011-01-13 15:10:00,0.0,15107,United Kingdom,False
86348,543599,84535B,FAIRY CAKES NOTEBOOK A6 SIZE,16,2011-02-10 13:08:00,0.0,17560,United Kingdom,False
129443,547417,22062,CERAMIC BOWL WITH LOVE HEART DESIGN,36,2011-03-23 10:25:00,0.0,13239,United Kingdom,False
138642,548318,22055,MINI CAKE STAND HANGING STRAWBERY,5,2011-03-30 12:45:00,0.0,13113,United Kingdom,False
144350,548871,22162,HEART GARLAND RUSTIC PADDED,2,2011-04-04 14:42:00,0.0,14410,United Kingdom,False


In [19]:
# Identify rows where:
# - 'Description' does not contain all-uppercase words,
# - 'CustomerID' is 'UNKNOWN',
# - 'UnitPrice' is zero (indicating free items)
Transactions_df[
    (~Transactions_df['Description'].str.contains(r'\b[A-Z]+\b', na=False)) &
    (Transactions_df['CustomerID'] == 'UNKNOWN') &
    (Transactions_df['UnitPrice'] == 0)
]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled
619,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,UNKNOWN,United Kingdom,False
1964,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,UNKNOWN,United Kingdom,False
1965,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,UNKNOWN,United Kingdom,False
1966,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,UNKNOWN,United Kingdom,False
1981,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,UNKNOWN,United Kingdom,False
1982,536550,85044,NaN,1,2010-12-01 14:34:00,0.0,UNKNOWN,United Kingdom,False
2018,536552,20950,NaN,1,2010-12-01 14:34:00,0.0,UNKNOWN,United Kingdom,False
2019,536553,37461,NaN,3,2010-12-01 14:35:00,0.0,UNKNOWN,United Kingdom,False
2020,536554,84670,NaN,23,2010-12-01 14:35:00,0.0,UNKNOWN,United Kingdom,False
4339,536765,84952C,NaN,19,2010-12-02 14:43:00,0.0,UNKNOWN,United Kingdom,False


In [20]:
pd.reset_option('display.max_rows')

In [21]:
# There's one thing to gift 0-priced items to unknown customers. But above's Description column is clearly stating that these rows do not mean anything and must be removed. 
Transactions_df = Transactions_df[
    ~(
        (~Transactions_df['Description'].str.contains(r'\b[A-Z]+\b', na=False)) &
        (Transactions_df['CustomerID'] == 'UNKNOWN') &
        (Transactions_df['UnitPrice'] == 0)
    )
]

## **7. Description**

In [22]:
# Missing values? 
print("It's okay to have missing values in Description as long as InvoiceNo and StockCode are not missing.\n")
Transactions_df[Transactions_df['Description'].isnull()]

It's okay to have missing values in Description as long as InvoiceNo and StockCode are not missing.



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled


In [23]:
# Do all instances of the same item share the same description? 
description_check = Transactions_df.groupby('StockCode')['Description'].nunique()

# StockCodes with more than one unique description
inconsistent_stockcodes = description_check[description_check > 1]

print(f"Number of inconsistent StockCodes: {len(inconsistent_stockcodes)}")

Number of inconsistent StockCodes: 225


In [24]:
print("\nExamples of inconsistent StockCodes:")
for stockcode in inconsistent_stockcodes.index[:10]:  # First 10 examples
    print(Transactions_df[Transactions_df['StockCode'] == stockcode][['StockCode', 'Description']].drop_duplicates())
    print("-" * 50)


Examples of inconsistent StockCodes:
       StockCode     Description
47060     16156L  WRAP, CAROUSEL
137274    16156L   WRAP CAROUSEL
--------------------------------------------------
       StockCode                          Description
6448      17107D  FLOWER FAIRY,5 SUMMER B'DRAW LINERS
313823    17107D         FLOWER FAIRY 5 DRAWER LINERS
345675    17107D    FLOWER FAIRY 5 SUMMER DRAW LINERS
--------------------------------------------------
       StockCode          Description
1853       20622   VIPPASSPORT COVER 
242245     20622  VIP PASSPORT COVER 
--------------------------------------------------
      StockCode              Description
92        20725  LUNCH BAG RED RETROSPOT
57853     20725     LUNCH BAG RED SPOTTY
--------------------------------------------------
      StockCode                          Description
412       20914  SET/5 RED RETROSPOT LID GLASS BOWLS
57842     20914     SET/5 RED SPOTTY LID GLASS BOWLS
-----------------------------------------------

In [25]:
# Function to get the most frequent description for each StockCode
def get_most_frequent_description(Descriptions):
    return Descriptions.value_counts().idxmax()

# Get the most frequent description for each StockCode
most_frequent_descriptions = Transactions_df.groupby('StockCode')['Description'].agg(get_most_frequent_description)

# Replace all 'Description' values with the most frequent one for their StockCode
Transactions_df['Description'] = Transactions_df['StockCode'].map(most_frequent_descriptions)

# Verify Results
description_check = Transactions_df.groupby('StockCode')['Description'].nunique()
inconsistent_stockcodes = description_check[description_check > 1]
print(f"Number of inconsistent StockCodes: {len(inconsistent_stockcodes)}")

Number of inconsistent StockCodes: 0


## **8. Country**

In [26]:
# Missing values? 
Transactions_df[Transactions_df['Country'].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled


In [27]:
# Invalid country names? 
print(Transactions_df['Country'].value_counts())

Country
United Kingdom          491769
Germany                   9096
France                    8234
EIRE                      8075
Spain                     2468
Netherlands               2330
Belgium                   1971
Switzerland               1969
Portugal                  1475
Australia                 1257
Norway                    1060
Italy                      783
Channel Islands            753
Finland                    653
Cyprus                     619
Unspecified                446
Sweden                     437
Austria                    387
Denmark                    375
Japan                      355
Poland                     336
Israel                     297
USA                        291
Hong Kong                  280
Singapore                  215
Iceland                    182
Canada                     150
Greece                     142
Malta                      123
United Arab Emirates        67
European Community          58
RSA                         57


In [28]:
Transactions_df.to_csv('Transactions.csv', index=False)